In [21]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)


In [22]:
from torchinfo import summary

summary(model, depth=7)

Layer (type:depth-idx)                                  Param #
DistilBertForMaskedLM                                   --
├─GELUActivation: 1-1                                   --
├─DistilBertModel: 1-2                                  --
│    └─Embeddings: 2-1                                  --
│    │    └─Embedding: 3-1                              23,440,896
│    │    └─Embedding: 3-2                              393,216
│    │    └─LayerNorm: 3-3                              1,536
│    │    └─Dropout: 3-4                                --
│    └─Transformer: 2-2                                 --
│    │    └─ModuleList: 3-5                             --
│    │    │    └─TransformerBlock: 4-1                  --
│    │    │    │    └─MultiHeadSelfAttention: 5-1       --
│    │    │    │    │    └─Dropout: 6-1                 --
│    │    │    │    │    └─Linear: 6-2                  590,592
│    │    │    │    │    └─Linear: 6-3                  590,592
│    │    │    │    │    

In [ ]:
# 質問の入力
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

device='cuda'


In [24]:
text = "This is a great [MASK]."

In [27]:
import torch
device='cuda'
inputs = tokenizer(text, return_tensors="pt")
inputs.to(device)
model.to(device)
token_logits = model(**inputs).logits
inputs['input_ids']

tensor([[ 101, 2023, 2003, 1037, 2307,  103, 1012,  102]], device='cuda:0')

In [28]:
tokenizer.mask_token_id

103

In [ ]:
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")